In [ ]:
from __future__ import print_function
import os,sys
import torch
import ROOT as rt
from larlite import larlite
from ROOT import larutil
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import lardly 

In [ ]:
# LOAD THE FILE/DATASET LOADER
tripletfile_v = ["../out.root"]

tree = rt.TChain("larmatchtriplet")
for f in tripletfile_v:
    tree.AddFile(f)
nentries = tree.GetEntries()
print("NENTRIES: ",nentries)

In [ ]:
# SETUP THE GEOMETRY

# Set the detector in larlite
#detid = larlite.geo.kICARUS
detid = larlite.geo.kMicroBooNE
#detid = larlite.geo.kSBND
larutil.LArUtilConfig.SetDetector(detid)

# Get geometry class
geo = larlite.larutil.Geometry.GetME()
detp = larutil.DetectorProperties.GetME()

# Get detector outlines from lardly
from lardly.detectors.getdetectoroutlines import DetectorOutlineFromLarlite
detoutline = DetectorOutlineFromLarlite(detid)
detlines = detoutline.getlines()

In [ ]:
# DEFINE PARTICLE ID COLORS
particle_id_color = {0:(0,0,0),      # no label
                     1:(255,125,50), # Cosmic
                     2:(0,0,0),      # BNB
                     3:(255,0,0),    # electron
                     4:(0,255,0),    # gamma
                     5:(0,125,125),  # pi0
                     6:(155,0,155),  # Muon
                     7:(255,255,0),  # Kaon
                     8:(255,165,0),  # pion                     
                     9:(0,0,255)}    # proton

ssnet_id_color = {0:(0,0,0),      # no label
                  1:(255,125,50), # electron
                  2:(0,255,0),    # gamma
                  3:(0,0,255),    # muon
                  4:(0,125,125),  # pion
                  5:(155,0,155),  # proton
                  6:(255,255,0)}  # other

particle_id_name = {0:"nolabel",  # no label
                    1:"delta",    # no label
                    2:"nolabel",  # no label
                    3:"electron", # electron
                    4:"gamma",    # gamma
                    5:"pi0",  # pi0
                    6:"muon", # Muon
                    7:"kaon", # Kaon
                    8:"pion", # proton
                    9:"proton"}   # pion

# define some colors for particle types
kp_color_array = np.array( ((255,0,0),    # nu
                            (0,255,0),    # track-start
                            (0,0,255),    # track-end
                            (255,0,255),  # shower
                            (0,255,255),  # shower-michel
                            (255,255,0)),     # shower-delta
                            dtype=np.float64 ) 

In [ ]:
# Get entry data
ENTRY = 0
tree.GetEntry(ENTRY)
tpcdata = tree.triplet_v.at(0)

In [ ]:
# Get triplet data numpy arrays
from ctypes import c_int
ntriplets = tpcdata._triplet_v.size()
print("num triplets: ",ntriplets)
nfilled = c_int(0)
tripdata = tpcdata.sample_triplet_matches(ntriplets,nfilled,True )
data_dict = tpcdata.make_spacepoint_charge_array()
print("spt_v array: ",data_dict.keys())
pos = data_dict['spacepoint_t'][:,:3]
matchtruth = tripdata[:,3]
print("num filled: ",nfilled)
print("pos array: ",pos.shape)
print("matchtruth array: ",matchtruth.shape)


In [ ]:
# get matchspan data numpy arrays
spandata = tpcdata.get_matchspan_array()
print("spandata array: ",spandata.shape)

In [ ]:
# PLOT SPACEPOINT TRUTH
TRUE_ONLY=False

sp_plots = []
truth =  data_dict['truetriplet_t']
if TRUE_ONLY:
    xpos = pos[truth==1,:]
    truth =  data_dict['truetriplet_t'][truth==1]
else:
    xpos = pos
ntruth = (truth==1).sum()
nfalse = (truth==0).sum()
print("ntruth=",ntruth," nfalse=",nfalse," ratio: ",float(ntruth)/float(nfalse))
    
plot = go.Scatter3d( x=xpos[:,0], y=xpos[:,1], z=xpos[:,2],
                    text=np.arange(0,xpos.shape[0]),
                    #hoverinfo="text",
                    mode="markers", name="", 
                    marker={"size":1.0,"opacity":0.5,"color":truth,"colorscale":"Viridis"} )  
sp_plots.append(plot)
    
axis_template = {
    "showbackground": True,
    #"backgroundcolor": "rgba(100, 100, 100,0.5)",
    "backgroundcolor": "rgba(0, 0, 0,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()

In [ ]:
# PLOT SPACEPOINT TRUTH
TRUE_ONLY=False
SAMPLE = False
CUT_ON_SPAN = True
NUM_SAMPLES = 100000

#SPANINDEX = 0 # max span
#SPANINDEX = 1 # min span
SPANINDEX = 2 # cycle span
SPAN_HINGE = 2
SPAN_LEN = 10.0

sp_plots = []
truth =  data_dict['truetriplet_t']

if TRUE_ONLY:
    xpos = pos[truth==1,:]
    truth =  data_dict['truetriplet_t'][truth==1]
    spanplot = spandata[truth==1][:,SPANINDEX]
else:
    xpos = pos
    spanplot = spandata[:,SPANINDEX]
    
if SAMPLE and xpos.shape[0]>NUM_SAMPLES:
    idx = np.arange(xpos.shape[0])
    np.random.shuffle(idx)
    xpos = xpos[idx[:NUM_SAMPLES],:]
    truth = truth[idx[:NUM_SAMPLES]]
    spanplot = spanplot[ idx[:NUM_SAMPLES] ]
  
spanplot = np.clip(spanplot-SPAN_HINGE,0,200)
spanplot = np.exp(-spanplot/SPAN_LEN )
colorplot = spanplot

if CUT_ON_SPAN:
    xpos = xpos[spanplot<0.5,:]
    truth = truth[spanplot<0.5]
    spanplot = spanplot[spanplot<0.5]
    colorplot = truth

ntruth = (truth==1).sum()
nfalse = (truth==0).sum()
print("ntruth=",ntruth," nfalse=",nfalse)
    
plot = go.Scatter3d( x=xpos[:,0], y=xpos[:,1], z=xpos[:,2],
                    text=np.arange(0,xpos.shape[0]),
                    mode="markers", name="", 
                    marker={"size":2.0,"opacity":0.5,
                            "color":colorplot,
                            "cmin":0.0,
                            "cmax":1.0,
                            "colorscale":"Viridis"} )  
sp_plots.append(plot)
    
axis_template = {
    "showbackground": True,
    #"backgroundcolor": "rgba(100, 100, 100,0.5)",
    "backgroundcolor": "rgba(0, 0, 0,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}


layout = go.Layout(
    title='DETECTOR TPC',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 2},
        "camera": {"eye": {"x": -3, "y": 0.1, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=sp_plots+detlines, layout=layout)
fig.show()